In [9]:
from importlib import reload
import pathlib
import os
import sys

In [10]:
this_path = pathlib.Path('.').parent.absolute()
src_path = os.path.join(this_path, '..')
sys.path.insert(0, src_path)

In [29]:
import dnashapeparams as dsp
import find_motifs as fm
import numpy as np
import inout

In [86]:
params = [
    "test/BRCA1_input/BRCA1_30_bp_height_train_4.fa.EP",
    "test/BRCA1_input/BRCA1_30_bp_height_train_4.fa.HelT",
    "test/BRCA1_input/BRCA1_30_bp_height_train_4.fa.MGW",
    "test/BRCA1_input/BRCA1_30_bp_height_train_4.fa.ProT",
    "test/BRCA1_input/BRCA1_30_bp_height_train_4.fa.Roll",
]
param_names = [
    "EP",
    "HelT",
    "MGW",
    "ProT",
    "Roll",
]
search_method = "greedy"
in_file = "test/BRCA1_input/BRCA1_30_bp_height_train_4.txt"
kmer = 15
ignorestart = 2
ignoreend = 2
threshold_perc = 0.05
threshold_seeds = 2.0
threshold_match = 2.0
seeds_per_seq_thresh = 1
seeds_per_seq = 1
num_seeds = 1000

In [17]:
all_params = [
    fm.read_parameter_file(x) for x in params
]

In [18]:
all_params

In [20]:
dir(all_params[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_entry',
 'chrm_names',
 'data',
 'names',
 'pull_entry',
 'read_whole_datafile',
 'read_whole_file',
 'write']

In [21]:
all_params[0].data

{'peak_8_0': <inout.FastaEntry at 0x7fc54d51fb80>,
 'peak_27_2': <inout.FastaEntry at 0x7fc54d51fd00>,
 'peak_403_0': <inout.FastaEntry at 0x7fc54d51fc40>,
 'peak_227_0': <inout.FastaEntry at 0x7fc54d51fb20>,
 'peak_79': <inout.FastaEntry at 0x7fc54d51f5b0>,
 'peak_33_1': <inout.FastaEntry at 0x7fc54d51f550>,
 'peak_369': <inout.FastaEntry at 0x7fc54d51f4c0>,
 'peak_165_1': <inout.FastaEntry at 0x7fc54d51f460>,
 'peak_129_2': <inout.FastaEntry at 0x7fc54d51f400>,
 'peak_367_2': <inout.FastaEntry at 0x7fc54d51f3a0>,
 'peak_212_1': <inout.FastaEntry at 0x7fc54d51f340>,
 'peak_547': <inout.FastaEntry at 0x7fc54d51f2e0>,
 'peak_508_1': <inout.FastaEntry at 0x7fc54d51f2b0>,
 'peak_129_0': <inout.FastaEntry at 0x7fc54e8c69d0>,
 'peak_118_2': <inout.FastaEntry at 0x7fc59354e310>,
 'peak_101_2': <inout.FastaEntry at 0x7fc59354e1c0>,
 'peak_116_0': <inout.FastaEntry at 0x7fc54d3f39d0>,
 'peak_376_0': <inout.FastaEntry at 0x7fc54d3f3970>,
 'peak_478_0': <inout.FastaEntry at 0x7fc54d3f31c0>,
 'pe

In [24]:
dir(all_params[0].data['peak_8_0'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'chrm_name',
 'header',
 'length',
 'pull_seq',
 'seq',
 'set_header',
 'set_seq',
 'write']

In [27]:
all_params[0].data['peak_8_0'].seq

[nan,
 nan,
 -5.32,
 -7.47,
 -8.32,
 -6.18,
 -7.05,
 -5.27,
 -5.25,
 -6.87,
 -6.9,
 -5.67,
 -5.71,
 -8.4,
 -8.76,
 -9.42,
 -8.22,
 -5.5,
 -5.54,
 -5.02,
 -6.56,
 -5.64,
 -6.03,
 -7.86,
 -7.97,
 -6.03,
 -7.69,
 -7.06,
 -5.24,
 -5.35,
 -5.45,
 -5.39,
 -5.25,
 -5.39,
 -5.02,
 -6.72,
 -5.52,
 -5.59,
 -7.61,
 -5.66,
 -6.85,
 -5.43,
 -5.36,
 -6.85,
 -5.43,
 -6.24,
 -9.12,
 -7.85,
 -7.41,
 -7.24,
 -7.66,
 -6.03,
 -5.59,
 -7.22,
 -5.52,
 -5.59,
 -7.22,
 -5.52,
 nan,
 nan]

In [28]:
this_dist = dsp.manhattan_distance

In [43]:
cats = inout.SeqDatabase(names=[])
cats.read(in_file, int)

In [44]:
cats

In [45]:
dir(cats)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'calculate_enrichment',
 'category_subset',
 'center_spread',
 'determine_center_spread',
 'discretize_RZ',
 'discretize_quant',
 'get_names',
 'get_values',
 'iterate_through_precompute',
 'joint_entropy',
 'mutual_information',
 'names',
 'normalize_params',
 'params',
 'pre_compute_windows',
 'random_subset',
 'random_subset_by_class',
 'read',
 'set_center_spread',
 'shannon_entropy',
 'shuffle',
 'unnormalize_params',
 'values',
 'vectors',
 'write']

In [46]:
cats.get_values()

array([0, 0, 0, ..., 0, 0, 0])

In [47]:
np.max(cats.get_values())

1

In [48]:
np.mean(cats.get_values())

0.24036281179138322

In [50]:
fm.seqs_per_bin(cats)

'\nCat 0: 1340\nCat 1: 424'

In [53]:
# add parameter values for each element of cats
for name, param in zip(cats.names, cats):
    # loop over shape parameter information, adding the parameter's info
    #  to this element of cats
    for this_param, this_param_name in zip(all_params, param_names):
        param.add_shape_param(
            dsp.ShapeParamSeq(
                this_param_name,
                this_param.pull_entry(name).seq,
            ),
        )
        param.metric = this_dist

In [54]:
cats[0]

IndexError: list index out of range

In [59]:
dir(cats.params[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_shape_param',
 'as_vector',
 'data',
 'distance',
 'from_vector',
 'matrix',
 'metric',
 'names',
 'normalize_values',
 'rev_comp',
 'sliding_windows',
 'unnormalize_values',
 'vector',
 'windows']

In [60]:
cats.params[0].names

['EP', 'HelT', 'MGW', 'ProT', 'Roll']

In [64]:
cats.params[0].data

{'EP': <dnashapeparams.ShapeParamSeq at 0x7fc54b8689a0>,
 'HelT': <dnashapeparams.ShapeParamSeq at 0x7fc54b868880>,
 'MGW': <dnashapeparams.ShapeParamSeq at 0x7fc54b868a60>,
 'ProT': <dnashapeparams.ShapeParamSeq at 0x7fc54b868a90>,
 'Roll': <dnashapeparams.ShapeParamSeq at 0x7fc54b868610>}

In [66]:
dir(cats.params[0].data["EP"])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_param',
 'get_values',
 'name',
 'normalize_values',
 'params',
 'rev_comp',
 'unnormalize_values']

In [67]:
cats.params[0].data["EP"].get_values()

array([  nan,   nan, -5.32, -7.47, -8.32, -6.18, -7.05, -5.27, -5.25,
       -6.87, -6.9 , -5.67, -5.71, -8.4 , -8.76, -9.42, -8.22, -5.5 ,
       -5.54, -5.02, -6.56, -5.64, -6.03, -7.86, -7.97, -6.03, -7.69,
       -7.06, -5.24, -5.35, -5.45, -5.39, -5.25, -5.39, -5.02, -6.72,
       -5.52, -5.59, -7.61, -5.66, -6.85, -5.43, -5.36, -6.85, -5.43,
       -6.24, -9.12, -7.85, -7.41, -7.24, -7.66, -6.03, -5.59, -7.22,
       -5.52, -5.59, -7.22, -5.52,   nan,   nan])

In [68]:
cats.determine_center_spread()
cats.normalize_params()

In [69]:
cats.params[0].data["EP"].get_values()

array([        nan,         nan,  0.85231105, -0.4660118 , -0.9872092 ,
        0.32498191, -0.20847896,  0.88296972,  0.89523319, -0.09810775,
       -0.11650295,  0.63770035,  0.61317342, -1.03626308, -1.25700551,
       -1.66169996, -0.92589186,  0.74193984,  0.7174129 ,  1.03626308,
        0.09197601,  0.65609556,  0.41695792, -0.70514943, -0.77259851,
        0.41695792, -0.60090995, -0.2146107 ,  0.90136492,  0.83391585,
        0.77259851,  0.80938891,  0.89523319,  0.80938891,  1.03626308,
       -0.00613173,  0.72967637,  0.68675423, -0.55185608,  0.64383209,
       -0.08584428,  0.78486197,  0.82778411, -0.08584428,  0.78486197,
        0.28819151, -1.47774794, -0.6990177 , -0.42922139, -0.32498191,
       -0.58251475,  0.41695792,  0.68675423, -0.31271844,  0.72967637,
        0.68675423, -0.31271844,  0.72967637,         nan,         nan])

In [75]:
# the vectors attribute contains a list of sliding windows for each
#  cat
cats.vectors

In [72]:
cats.pre_compute_windows(
    wsize = kmer,
    wstart = ignorestart,
    wend = ignoreend
)

In [76]:
cats.vectors

[[<dnashapeparams.ShapeParams at 0x7fc54b0aa4f0>,
 ...]

In [81]:
mean, stdev = fm.find_initial_threshold(
    cats.random_subset_by_class(
        threshold_perc,
    ),
    seeds_per_seq_thresh,
)
threshold_seeds = max(mean - threshold_seeds*stdev, 0)
threshold_match = max(mean - threshold_match*stdev, 0)

In [82]:
threshold_seeds

57.815940822257495

In [83]:
threshold_match

57.815940822257495

In [87]:
possible_motifs = fm.greedy_search2(cats, threshold_seeds, num_seeds, seeds_per_seq)

In [89]:
len(possible_motifs)

1000

In [90]:
possible_motifs

In [92]:
this_cats = cats
other_cats = cats

For each potential motif, the evaluate seeds function compares the potential motif to each sequence in the ground truth data to determine whether it has a match to the potential motif. If it does, the index for this sequence in the ground truth data is given a 1. Otherwise its index is given a 0.

Then, the ground truth vector is compared to this match vector to calculate their mutual information.

BUT, HOW IS THIS IMPLEMENTED FOR DISCRETIZED CONTINUOUS INPUT DATA? HERE, THE SEED MATCH VECTOR IS BINARY, NO MORE THAN TWO CATEGORIES!!

In [ ]:
all_seeds = evaluate_seeds(
    this_cats,
    possible_motifs,
    threshold_match,
    False,
)